## 1. 인공신경망 학습 코드 (Neural Network Training Code)

### 기본 설정 (Basic Setup)

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 재현성을 위한 시드 설정
torch.manual_seed(42)
np.random.seed(42)

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스 (Using device): {device}")

### 토이 데이터셋 생성 (Create Toy Dataset)

In [ ]:
# 2클래스 분류 문제를 위한 토이 데이터 생성
def generate_toy_data(n_samples=1000):
    # 두 개의 원형 클러스터 생성
    n_samples_per_class = n_samples // 2
    
    # 클래스 0 데이터 
    X0 = np.random.randn(n_samples_per_class, 2) * 0.5 + np.array([2, 2])
    y0 = np.zeros(n_samples_per_class)
    
    # 클래스 1 데이터
    X1 = np.random.randn(n_samples_per_class, 2) * 0.5 + np.array([-2, -2])
    y1 = np.ones(n_samples_per_class)
    
    # 데이터 합치기
    X = np.vstack([X0, X1])
    y = np.hstack([y0, y1])
    
    # 데이터 섞기
    indices = np.random.permutation(n_samples)
    X, y = X[indices], y[indices]
    
    return X, y

# 데이터 생성
X, y = generate_toy_data(1000)

In [ ]:
# 데이터 시각화
plt.figure(figsize=(8, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], color='blue', label='클래스 0')
plt.scatter(X[y==1, 0], X[y==1, 1], color='red', label='클래스 1')
plt.title('분류 문제를 위한 토이 데이터셋 ')
plt.xlabel('특성 1 (Feature 1)')
plt.ylabel('특성 2 (Feature 2)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 데이터를 학습/테스트로 분할
train_ratio = 0.8
train_size = int(len(X) * train_ratio)

X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# PyTorch 텐서로 변환
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

print(f"학습 데이터 크기 (Training data size): {X_train_tensor.shape}")
print(f"테스트 데이터 크기 (Test data size): {X_test_tensor.shape}")

### 신경망 모델 정의 (Define Neural Network Model)

In [ ]:
class SimpleNN(nn.Module):
    """
    간단한 피드포워드 신경망 (Simple feedforward neural network)
    """
    def __init__(self, input_size, hidden_size, output_size):
        """
        모델 구조 초기화 (Initialize model architecture)
        """
        super(SimpleNN, self).__init__()
        
        # 레이어 정의 (Define layers)
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.activation1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.activation2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()  # 이진 분류를 위한 시그모이드 (Sigmoid for binary classification)
        
    def forward(self, x):
        """
        순전파(Forward pass, 역전파는 자동계산, backward() 호출로 수행)
        """
        x = self.layer1(x)
        x = self.activation1(x)
        x = self.layer2(x)
        x = self.activation2(x)
        x = self.layer3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# 모델 인스턴스 생성 (Create model instance)
input_size = 2        # 입력 특성 수 (Number of input features)
hidden_size = 16      # 은닉층 크기 (Hidden layer size)
output_size = 1       # 출력 크기 (이진 분류) (Output size - binary classification)

model = SimpleNN(input_size, hidden_size, output_size).to(device)
print(model)

In [ ]:
# 손실 함수와 옵티마이저 정의 (Define loss function and optimizer)
criterion = nn.BCELoss()  # 이진 교차 엔트로피 손실 (Binary Cross Entropy Loss)
optimizer = optim.Adam(model.parameters(), lr=0.01)

### 학습 루프 (Training Loop)

In [ ]:
# 학습 과정 시각화를 위한 손실 기록 (Record losses for visualization)
losses = []

# 에포크 수 정의 (Define number of epochs)
num_epochs = 100

# 학습 루프 (Training loop)
for epoch in range(num_epochs):
    # 모델을 학습 모드로 설정 (Set model to training mode)
    model.train()
    
    # 순전파 (Forward pass)
    outputs = model(X_train_tensor)
    outputs = outputs.squeeze()  # 차원 축소 (Reduce dimensions)
    
    # 손실 계산 (Calculate loss)
    loss = criterion(outputs, y_train_tensor)
    
    # 역전파 및 최적화 (Backpropagation and optimization)
    optimizer.zero_grad()  # 기울기 초기화 (Zero gradients)
    loss.backward()        # 역전파 (Backpropagation)
    optimizer.step()       # 가중치 업데이트 (Update weights)
    
    # 손실 기록 (Record loss)
    losses.append(loss.item())
    
    # 학습 과정 출력 (Print training progress)
    if (epoch+1) % 10 == 0:
        # 테스트 데이터에 대한 성능 평가 (Evaluate on test data)
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor).squeeze()
            test_loss = criterion(test_outputs, y_test_tensor)
            
            # 예측값 계산 (Calculate predictions)
            train_preds = (outputs > 0.5).float()
            test_preds = (test_outputs > 0.5).float()
            
            # 정확도 계산 (Calculate accuracy)
            train_acc = (train_preds == y_train_tensor).float().mean()
            test_acc = (test_preds == y_test_tensor).float().mean()
            
        print(f'에포크 (Epoch) [{epoch+1}/{num_epochs}], '
              f'손실 (Loss): {loss.item():.4f}, '
              f'학습 정확도 (Train Acc): {train_acc:.4f}, '
              f'테스트 정확도 (Test Acc): {test_acc:.4f}')

### 학습 결과 시각화 (Visualize Training Results)

In [ ]:
# 손실 곡선 그리기
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('학습 중 손실 변화')
plt.xlabel('에포크 (Epoch)')
plt.ylabel('손실 (Loss)')
plt.grid(True)
plt.show()

In [ ]:
# 결정 경계 시각화 (Visualize decision boundary)
def plot_decision_boundary(model, X, y):
    # 그리드 생성 (Create a grid)
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    
    # 그리드 포인트에 대한 예측 (Predictions for grid points)
    model.eval()
    with torch.no_grad():
        grid = torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()]).to(device)
        Z = model(grid).squeeze().cpu().numpy()
        Z = (Z > 0.5).astype(int)
    
    # 결정 경계 표시 (Display decision boundary)
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
    
    # 데이터 포인트 표시 (Display data points)
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='클래스 0')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='클래스 1')
    
    plt.title('신경망 분류기의 결정 경계 (Decision Boundary of Neural Network Classifier)')
    plt.xlabel('특성 1 (Feature 1)')
    plt.ylabel('특성 2 (Feature 2)')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_decision_boundary(model, X, y)

In [ ]:
# 모델 평가 (Evaluate model)
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor).squeeze()
    test_preds = (test_outputs > 0.5).float()
    test_acc = (test_preds == y_test_tensor).float().mean()
    
print(f'최종 테스트 정확도 (Final Test Accuracy): {test_acc:.4f}')

## 2. 텍스트 인코딩 (토큰화, 임베딩) (Text Encoding - Tokenization, Embedding)

### 토이 텍스트 데이터 생성 (Create Toy Text Data)

In [ ]:
# 간단한 텍스트 분류 데이터
texts = [
    "이 영화는 정말 재미있었어요",
    "저는 이 책을 매우 좋아했습니다",
    "음식이 맛있지 않았어요",  
    "서비스가 나빴습니다",  
    "이 제품은 품질이 좋습니다",
    "가격이 너무 비쌉니다",
    "이 장소는 아름다웠어요",
    "날씨가 좋지 않았어요",
    "직원들이 친절했습니다",
    "대기 시간이 너무 길었어요",
]

# 라벨 생성 (긍정: 1, 부정: 0)
labels = [1, 1, 0, 0, 1, 0, 1, 0, 1, 0]

# 데이터 출력
for text, label in zip(texts, labels):
    sentiment = "긍정(Positive)" if label == 1 else "부정(Negative)"
    print(f"텍스트 (Text): {text} | 감정 (Sentiment): {sentiment}")

### 토큰화 (Tokenization)

In [ ]:
def tokenize(text):
    """
    간단한 공백 기반 토큰화 (Simple space-based tokenization)
    """
    # 소문자로 변환 (Convert to lowercase)
    text = text.lower()
    # 기본 전처리 (Basic preprocessing)
    for char in '.,!?':
        text = text.replace(char, ' ' + char + ' ')
    # 토큰 분리 (Split tokens)
    tokens = text.split()
    return tokens

tokenized_texts = [tokenize(text) for text in texts]

# 토큰화 결과 출력 (Print tokenization results)
for i, tokens in enumerate(tokenized_texts[:3]):  # 처음 3개만 출력 (Print first 3 only)
    print(f"원본 (Original): {texts[i]}")
    print(f"토큰화 (Tokenized): {tokens}")
    print("-" * 50)

In [ ]:
# 어휘 사전 구축 (Build vocabulary)
def build_vocab(tokenized_texts):
    """
    토큰화된 텍스트에서 어휘 사전 생성 (Create vocabulary from tokenized texts)
    """
    # 모든 고유 토큰 수집 (Collect all unique tokens)
    vocab = set()
    for tokens in tokenized_texts:
        vocab.update(tokens)
    
    # 사전 생성 (Create dictionary)
    word_to_idx = {word: idx+1 for idx, word in enumerate(sorted(vocab))}
    # 패딩을 위한 0 추가 (Add 0 for padding)
    word_to_idx['<PAD>'] = 0
    # 알 수 없는 토큰을 위한 인덱스 추가 (Add index for unknown tokens)
    word_to_idx['<UNK>'] = len(word_to_idx)
    
    # 역 매핑 사전 생성 (Create reverse mapping)
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    
    return word_to_idx, idx_to_word

# 어휘 사전 생성 (Create vocabulary)
word_to_idx, idx_to_word = build_vocab(tokenized_texts)

print(f"어휘 크기 (Vocabulary size): {len(word_to_idx)}")
print(f"어휘 사전 샘플 (Vocabulary sample): {dict(list(word_to_idx.items())[:10])}")

In [ ]:
# 텍스트를 인덱스로 변환 (Convert text to indices)
def text_to_indices(tokenized_text, word_to_idx, max_len=10):
    """
    토큰화된 텍스트를 인덱스 리스트로 변환 (Convert tokenized text to list of indices)
    """
    indices = []
    
    # 각 토큰을 인덱스로 변환 (Convert each token to index)
    for token in tokenized_text[:max_len]:
        if token in word_to_idx:
            indices.append(word_to_idx[token])
        else:
            indices.append(word_to_idx['<UNK>'])
    
    # 패딩 추가 (Add padding)
    while len(indices) < max_len:
        indices.append(word_to_idx['<PAD>'])
    
    return indices

# 모든 텍스트를 인덱스로 변환 (Convert all texts to indices)
max_len = 10  # 최대 시퀀스 길이 (Maximum sequence length)
indices_list = [text_to_indices(tokens, word_to_idx, max_len) for tokens in tokenized_texts]

In [ ]:
# 결과 출력 (Print results)
for i in range(3):  # 처음 3개만 출력 (Print first 3 only)
    print(f"원본 (Original): {texts[i]}")
    print(f"토큰화 (Tokenized): {tokenized_texts[i]}")
    print(f"인덱스 (Indices): {indices_list[i]}")
    print("-" * 50)

# 인덱스를 PyTorch 텐서로 변환 (Convert indices to PyTorch tensor)
X_tensor = torch.LongTensor(indices_list).to(device)
y_tensor = torch.FloatTensor(labels).to(device)

print(f"X 텐서 모양 (X tensor shape): {X_tensor.shape}")
print(f"y 텐서 모양 (y tensor shape): {y_tensor.shape}")

### 임베딩 모델 (Embedding Model)

In [ ]:
class TextClassifier(nn.Module):
    """
    임베딩 레이어를 사용한 텍스트 분류 모델 (Text classification model using embedding layer)
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        """
        모델 초기화 (Initialize model)
        """
        super(TextClassifier, self).__init__()
        
        # 임베딩 레이어 (Embedding layer)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # LSTM 레이어 (LSTM layer)
        self.lstm = nn.LSTM(embedding_dim, 
                          hidden_dim, 
                          num_layers=1, 
                          batch_first=True, 
                          bidirectional=True)
        
        # 드롭아웃 (Dropout)
        self.dropout = nn.Dropout(0.3)
        
        # 출력 레이어 (Output layer)
        # 양방향 LSTM이므로 hidden_dim * 2 (Bidirectional LSTM, so hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        # 활성화 함수 (Activation function)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text):
        """
        순전파 (Forward pass)
        """
        # 텍스트를 임베딩으로 변환 (Convert text to embeddings)
        # text: [batch_size, seq_len]
        embedded = self.embedding(text)
        # embedded: [batch_size, seq_len, embedding_dim]
        
        # LSTM 레이어에 통과 (Pass through LSTM layer)
        output, (hidden, cell) = self.lstm(embedded)
        # output: [batch_size, seq_len, hidden_dim * 2]
        # hidden: [2, batch_size, hidden_dim]
        
        # 양방향 LSTM의 마지막 은닉 상태 연결 (Concatenate last hidden states of bidirectional LSTM)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden: [batch_size, hidden_dim * 2]
        
        # 드롭아웃 적용 (Apply dropout)
        hidden = self.dropout(hidden)
        
        # 출력층에 통과 (Pass through output layer)
        output = self.fc(hidden)
        # output: [batch_size, output_dim]
        
        # 시그모이드 활성화 (Sigmoid activation)
        return self.sigmoid(output)

# 모델 인스턴스 생성 (Create model instance)
vocab_size = len(word_to_idx)
embedding_dim = 50
hidden_dim = 32
output_dim = 1

model = TextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
print(model)

# 손실 함수와 옵티마이저 정의 (Define loss function and optimizer)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### 텍스트 모델 학습 (Train Text Model)

In [ ]:
# 학습/테스트 데이터 분할 (Split training/test data)
def train_test_split(X, y, train_ratio=0.8):
    """
    데이터를 학습/테스트로 분할 (Split data into train/test)
    """
    n_samples = len(X)
    train_size = int(n_samples * train_ratio)
    
    # 데이터 인덱스 섞기 (Shuffle data indices)
    indices = torch.randperm(n_samples)
    
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    return X_train, y_train, X_test, y_test

# 데이터 분할 (Split data)
X_train, y_train, X_test, y_test = train_test_split(X_tensor, y_tensor)

print(f"학습 데이터 크기 (Training data size): {X_train.shape}")
print(f"테스트 데이터 크기 (Test data size): {X_test.shape}")

In [ ]:
# 학습 루프 (Training loop)
num_epochs = 100
losses = []

for epoch in range(num_epochs):
    # 모델을 학습 모드로 설정 (Set model to training mode)
    model.train()
    
    # 순전파 (Forward pass)
    outputs = model(X_train).squeeze()
    
    # 손실 계산 (Calculate loss)
    loss = criterion(outputs, y_train)
    
    # 역전파 및 최적화 (Backpropagation and optimization)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 손실 기록 (Record loss)
    losses.append(loss.item())
    
    # 학습 과정 출력 (Print training progress)
    if (epoch + 1) % 10 == 0:
        # 테스트 데이터에 대한 성능 평가 (Evaluate on test data)
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test).squeeze()
            test_loss = criterion(test_outputs, y_test)
            
            # 예측값 계산 (Calculate predictions)
            train_preds = (outputs > 0.5).float()
            test_preds = (test_outputs > 0.5).float()
            
            # 정확도 계산 (Calculate accuracy)
            train_acc = (train_preds == y_train).float().mean()
            test_acc = (test_preds == y_test).float().mean()
        
        print(f'에포크 (Epoch) [{epoch+1}/{num_epochs}], '
              f'손실 (Loss): {loss.item():.4f}, '
              f'학습 정확도 (Train Acc): {train_acc:.4f}, '
              f'테스트 정확도 (Test Acc): {test_acc:.4f}')

### 모델 평가 및 임베딩 시각화 (Evaluate Model and Visualize Embeddings)

In [ ]:
# 손실 곡선 그리기 (Plot loss curve)
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('학습 중 손실 변화 (Loss During Training)')
plt.xlabel('에포크 (Epoch)')
plt.ylabel('손실 (Loss)')
plt.grid(True)
plt.show()

In [ ]:
# 임베딩 가중치 추출 (Extract embedding weights)
embedding_weights = model.embedding.weight.cpu().detach().numpy()
print(f"임베딩 가중치 모양 (Embedding weights shape): {embedding_weights.shape}")

In [ ]:
# 임베딩 시각화를 위한 차원 축소 (Dimensionality reduction for embedding visualization)
from sklearn.decomposition import PCA

# PCA를 사용한 차원 축소 (Dimensionality reduction using PCA)
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embedding_weights)

In [ ]:
# 패딩과 UNK 토큰 제외 (Exclude padding and UNK tokens)
valid_embeddings = embeddings_2d[1:-1]  # 패딩과 UNK 제외 (Exclude padding and UNK)
valid_words = list(word_to_idx.keys())[1:-1]  # 패딩과 UNK 제외 (Exclude padding and UNK)

In [ ]:
plt.title('단어 임베딩 PCA 시각화 (Word Embeddings PCA Visualization)')
plt.xlabel('주성분 1 (Principal Component 1)')
plt.ylabel('주성분 2 (Principal Component 2)')
plt.grid(True)
plt.show()

### 새로운 텍스트 분류 (Classify New Text)

In [ ]:
# 새로운 텍스트 분류 함수 (Function to classify new text)
def classify_text(text, model, word_to_idx, max_len=10):
    """
    새로운 텍스트 분류 (Classify new text)
    """
    # 토큰화 (Tokenize)
    tokens = tokenize(text)
    
    # 인덱스 변환 (Convert to indices)
    indices = text_to_indices(tokens, word_to_idx, max_len)
    
    # 텐서 변환 (Convert to tensor)
    tensor = torch.LongTensor([indices]).to(device)
    
    # 예측 (Predict)
    model.eval()
    with torch.no_grad():
        output = model(tensor).item()
    
    # 결과 해석 (Interpret result)
    probability = output
    prediction = 1 if probability > 0.5 else 0
    sentiment = "긍정(Positive)" if prediction == 1 else "부정(Negative)"
    
    return sentiment, probability

In [ ]:
# 새로운 텍스트 테스트 (Test new text)
new_texts = [
    "이 상품은 매우 좋아요",  
    "서비스가 형편없었습니다", 
    "생각보다 나쁘지 않았어요",
    "전반적으로 만족스럽습니다"
]

for text in new_texts:
    sentiment, probability = classify_text(text, model, word_to_idx)
    print(f"텍스트 (Text): {text}")
    print(f"감정 (Sentiment): {sentiment}")
    print(f"확률 (Probability): {probability:.4f}")
    print("-" * 50)